# Setup & Definitions
Here we import the necessary libraries and define the classes and the functions.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch.optim import Adam, SGD
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from itertools import cycle
import itertools

In [2]:
# To save the outputs
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
drive_results_path = '/content/drive/MyDrive/interleaved_training_results/resnet18'

In [10]:
#***MODELS & RELATED CLASSES***
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = F.relu(out)
        return out

class ResNet18TwoHeads(nn.Module):
    def __init__(self, num_classes1=50, num_classes2=50):
        super(ResNet18TwoHeads, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, 256)
        self.head1 = nn.Linear(256, num_classes1)  # First head
        self.head2 = nn.Linear(256, num_classes2)  # Second head

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * BasicBlock.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion),
            )

        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * BasicBlock.expansion
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc(x))
        out1 = self.head1(x)
        out2 = self.head2(x)
        return out1, out2

class ResNet18TwoHeads_MNIST(nn.Module):
    def __init__(self, num_classes1=5, num_classes2=5):
        super(ResNet18TwoHeads_MNIST, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, 256)
        self.head1 = nn.Linear(256, num_classes1)  # First head
        self.head2 = nn.Linear(256, num_classes2)  # Second head

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * BasicBlock.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion),
            )

        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * BasicBlock.expansion
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc(x))
        out1 = self.head1(x)
        out2 = self.head2(x)
        return out1, out2

# A single block for a CNN with residual connections
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # Two convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Downsample the input feature map to match dimensions if necessary
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels),
            )
        else:
            self.downsample = None

    # Forward pass of the residual block
    def forward(self, x):
        identity = x # Save the input value for the skip connection

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        # Apply downsampling if required
        if self.downsample is not None:
            identity = self.downsample(identity)
        out += identity

        return F.relu(out)

# Architecture for 32x32 RGB input (CIFAR10 and SVHN Dataset)
class MultiHeadCNN(nn.Module):
    def __init__(self):
        super(MultiHeadCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.resblock1 = ResidualBlock(32, 64, stride=2)
        self.resblock2 = ResidualBlock(64, 128, stride=2)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.head1 = nn.Linear(256, 5)  # First head (classes 0-4)
        self.head2 = nn.Linear(256, 5)  # Second head (classes 5-9)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.fc1(x))
        out1 = self.head1(x)
        out2 = self.head2(x)
        return out1, out2

# Architecture for 96x96 RGB input (STL-10 Dataset)
class MultiHeadCNN_96(nn.Module):
    def __init__(self):
        super(MultiHeadCNN_96, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.resblock1 = ResidualBlock(32, 64, stride=2)
        self.resblock2 = ResidualBlock(64, 128, stride=2)
        self.fc1 = nn.Linear(128 * 24 * 24, 256)
        self.head1 = nn.Linear(256, 5)  # First head (classes 0-4)
        self.head2 = nn.Linear(256, 5)  # Second head (classes 5-9)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = x.view(-1, 128 * 24 * 24)
        x = F.relu(self.fc1(x))
        out1 = self.head1(x)
        out2 = self.head2(x)
        return out1, out2

# Architecture for  28x28 grayscale input (MNIST/FashionMNIST datasets)
class MultiHeadCNN_MNIST(nn.Module):
    def __init__(self):
        super(MultiHeadCNN_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 64)
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.head1 = nn.Linear(256, 5)  # First head (classes 0-4)
        self.head2 = nn.Linear(256, 5)  # Second head (classes 5-9)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)  # Reduce to 14x14
        x = self.resblock1(x)
        x = F.max_pool2d(x, 2)  # Reduce to 7x7
        x = self.resblock2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        out1 = self.head1(x)
        out2 = self.head2(x)
        return out1, out2



#***TRAIN AND TRAINING LOOP FUNCTIONS***

def train(model, device, train_loader, optimizer, active_head, val_loader1, val_loader2):
    model.train()
    batch_losses = []
    batch_accuracies = []
    val_accuracies_1 = []
    val_accuracies_2 = []

    # Create an iterable that cycles through the validation data
    # We evaluate a batch obtained from this here as validation through each step
    val_iter1 = cycle(val_loader1)
    val_iter2 = cycle(val_loader2)

    for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc="Training Progress")):
        data, target = data.to(device), target.to(device)

        # Perform forward pass
        output1, output2 = model(data)
        if active_head == "head1":
            output = output1
        else:
            output = output2
            target -= 5  # Adjust labels for the second head

        # Calculate accuracy
        pred = output.argmax(dim=1, keepdim=True)
        correct = pred.eq(target.view_as(pred)).sum().item()
        accuracy = 100. * correct / target.size(0)
        batch_accuracies.append(accuracy)

        # Compute loss and backpropagate
        loss = torch.nn.functional.cross_entropy(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())


        model.eval()
        # Head 1
        val_data1, val_target1 = next(val_iter1)
        val_data1, val_target1 = val_data1.to(device), val_target1.to(device)
        val_output1, _ = model(val_data1)
        val_pred1 = val_output1.argmax(dim=1, keepdim=True)
        val_correct1 = val_pred1.eq(val_target1.view_as(val_pred1)).sum().item()
        val_total1 = val_target1.size(0)
        val_accuracy1 = 100. * val_correct1 / val_total1
        val_accuracies_1.append(val_accuracy1)

        # Head 2
        val_data2, val_target2 = next(val_iter2)
        val_data2, val_target2 = val_data2.to(device), val_target2.to(device)
        _, val_output2 = model(val_data2)
        val_target2 -= 5  # Adjust labels for the second head
        val_pred2 = val_output2.argmax(dim=1, keepdim=True)
        val_correct2 = val_pred2.eq(val_target2.view_as(val_pred2)).sum().item()
        val_total2 = val_target2.size(0)
        val_accuracy2 = 100. * val_correct2 / val_total2
        val_accuracies_2.append(val_accuracy2)

        model.train()

    return batch_losses, batch_accuracies, val_accuracies_1, val_accuracies_2



# Iterating through epochs
def training_loop(model, optimizer, run_length, total_epochs, experiment_name):
  for epoch in range(1, total_epochs + 1):
    if ((epoch-1) % (2 * run_length)) < run_length:
        print(f'On head 1, epoch {epoch}...')
        losses, accuracies, val_accuracies_1, val_accuracies_2 = train(model, device, loader_head1, optimizer, "head1", val_loader1 = loader_head1_test, val_loader2 = loader_head2_test)
        head = 1
        color = 'green'
    else:
        print(f'On head 2, epoch {epoch}...')
        losses, accuracies, val_accuracies_1, val_accuracies_2 = train(model, device, loader_head2, optimizer, "head2", val_loader1 = loader_head1_test, val_loader2 = loader_head2_test)
        head = 2
        color = 'orange'

    all_losses.extend(losses)
    all_accuracies.extend(accuracies)
    all_accuracies_val1.extend(val_accuracies_1)
    all_accuracies_val2.extend(val_accuracies_2)
    batch_heads.extend([head] * len(losses))
    colors.extend([color] * len(losses))

  data = {
      'all_losses': all_losses,
      'all_accuracies': all_accuracies,
      'all_accuracies_val1': all_accuracies_val1,
      'all_accuracies_val2': all_accuracies_val2,
      'batch_heads': batch_heads,
      'colors': colors
  }

  # Define the file path
  file_path = drive_results_path + f'/{experiment_name}.json'

  # Write the dictionary to a file as JSON
  with open(file_path, 'w') as file:
      json.dump(data, file)

def train_iid(model, device, data, target, optimizer, active_head, val_loader1, val_loader2):
    model.train()
    batch_losses = []
    batch_accuracies = []
    val_accuracies_1 = []
    val_accuracies_2 = []

    # Create an iterable that cycles through the validation data
    # We evaluate a batch obtained from this here as validation through each step
    val_iter1 = cycle(val_loader1)
    val_iter2 = cycle(val_loader2)
    data, target = data.to(device), target.to(device)


    # Perform forward pass
    output1, output2 = model(data)
    if active_head == "head1":
        output = output1
    else:
        output = output2
        target -= 5  # Adjust labels for the second head

    # Calculate accuracy
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()
    accuracy = 100. * correct / target.size(0)
    batch_accuracies.append(accuracy)

    # Compute loss and backpropagate
    loss = torch.nn.functional.cross_entropy(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    batch_losses.append(loss.item())


    model.eval()
    # Head 1
    val_data1, val_target1 = next(val_iter1)
    val_data1, val_target1 = val_data1.to(device), val_target1.to(device)
    val_output1, _ = model(val_data1)
    val_pred1 = val_output1.argmax(dim=1, keepdim=True)
    val_correct1 = val_pred1.eq(val_target1.view_as(val_pred1)).sum().item()
    val_total1 = val_target1.size(0)
    val_accuracy1 = 100. * val_correct1 / val_total1
    val_accuracies_1.append(val_accuracy1)

    # Head 2
    val_data2, val_target2 = next(val_iter2)
    val_data2, val_target2 = val_data2.to(device), val_target2.to(device)
    _, val_output2 = model(val_data2)
    val_target2 -= 5  # Adjust labels for the second head
    val_pred2 = val_output2.argmax(dim=1, keepdim=True)
    val_correct2 = val_pred2.eq(val_target2.view_as(val_pred2)).sum().item()
    val_total2 = val_target2.size(0)
    val_accuracy2 = 100. * val_correct2 / val_total2
    val_accuracies_2.append(val_accuracy2)

    model.train()

    return batch_losses, batch_accuracies, val_accuracies_1, val_accuracies_2

# Iterating through epochs for iid setting
def training_loop_iid(model, optimizer, run_length, total_epochs, experiment_name):
  for epoch in range(1, total_epochs + 1):
      print(f"Epoch {epoch}...")
      # Create iterators for both loaders
      iter_head1 = iter(loader_head1)
      iter_head2 = iter(loader_head2)

      # Cycle through both iterators
      for (data1, target1), (data2, target2) in itertools.zip_longest(iter_head1, iter_head2, fillvalue=(None, None)):
          if data1 is not None:
            losses, accuracies, val_accuracies_1, val_accuracies_2 = train_iid(model, device, data1, target1, optimizer, "head1", val_loader1 = loader_head1_test, val_loader2 = loader_head2_test)
            head = 1
            color = 'green'
            all_losses.extend(losses)
            all_accuracies.extend(accuracies)
            all_accuracies_val1.extend(val_accuracies_1)
            all_accuracies_val2.extend(val_accuracies_2)
            batch_heads.extend([head] * len(losses))
            colors.extend([color] * len(losses))
          if data2 is not None:
            losses, accuracies, val_accuracies_1, val_accuracies_2 = train_iid(model, device, data2, target2, optimizer, "head2", val_loader1 = loader_head1_test, val_loader2 = loader_head2_test)
            head = 2
            color = 'orange'
            all_losses.extend(losses)
            all_accuracies.extend(accuracies)
            all_accuracies_val1.extend(val_accuracies_1)
            all_accuracies_val2.extend(val_accuracies_2)
            batch_heads.extend([head] * len(losses))
            colors.extend([color] * len(losses))


  data = {
      'all_losses': all_losses,
      'all_accuracies': all_accuracies,
      'all_accuracies_val1': all_accuracies_val1,
      'all_accuracies_val2': all_accuracies_val2,
      'batch_heads': batch_heads,
      'colors': colors
  }

  # Define the file path
  file_path = drive_results_path + f'/{experiment_name}.json'

  # Write the dictionary to a file as JSON
  with open(file_path, 'w') as file:
      json.dump(data, file)


# Function to count the number of parameters in a model
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params



def train_100labels(model, device, train_loader, optimizer, active_head, val_loader1, val_loader2):
    model.train()
    batch_losses = []
    batch_accuracies = []
    val_accuracies_1 = []
    val_accuracies_2 = []

    # Create an iterable that cycles through the validation data
    # We evaluate a batch obtained from this here as validation through each step
    val_iter1 = cycle(val_loader1)
    val_iter2 = cycle(val_loader2)

    for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc="Training Progress")):
        data, target = data.to(device), target.to(device)

        # Perform forward pass
        output1, output2 = model(data)
        if active_head == "head1":
            output = output1
        else:
            output = output2
            target -= 50  # Adjust labels for the second head

        # Calculate accuracy
        pred = output.argmax(dim=1, keepdim=True)
        correct = pred.eq(target.view_as(pred)).sum().item()
        accuracy = 100. * correct / target.size(0)
        batch_accuracies.append(accuracy)

        # Compute loss and backpropagate
        loss = torch.nn.functional.cross_entropy(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())


        model.eval()
        # Head 1
        val_data1, val_target1 = next(val_iter1)
        val_data1, val_target1 = val_data1.to(device), val_target1.to(device)
        val_output1, _ = model(val_data1)
        val_pred1 = val_output1.argmax(dim=1, keepdim=True)
        val_correct1 = val_pred1.eq(val_target1.view_as(val_pred1)).sum().item()
        val_total1 = val_target1.size(0)
        val_accuracy1 = 100. * val_correct1 / val_total1
        val_accuracies_1.append(val_accuracy1)

        # Head 2
        val_data2, val_target2 = next(val_iter2)
        val_data2, val_target2 = val_data2.to(device), val_target2.to(device)
        _, val_output2 = model(val_data2)
        val_target2 -= 50  # Adjust labels for the second head
        val_pred2 = val_output2.argmax(dim=1, keepdim=True)
        val_correct2 = val_pred2.eq(val_target2.view_as(val_pred2)).sum().item()
        val_total2 = val_target2.size(0)
        val_accuracy2 = 100. * val_correct2 / val_total2
        val_accuracies_2.append(val_accuracy2)

        model.train()

    return batch_losses, batch_accuracies, val_accuracies_1, val_accuracies_2

# Iterating through epochs
def training_loop_100labels(model, optimizer, run_length, total_epochs, experiment_name):
  for epoch in range(1, total_epochs + 1):
    if ((epoch-1) % (2 * run_length)) < run_length:
        print(f'On head 1, epoch {epoch}...')
        losses, accuracies, val_accuracies_1, val_accuracies_2 = train_100labels(model, device, loader_head1, optimizer, "head1", val_loader1 = loader_head1_test, val_loader2 = loader_head2_test)
        head = 1
        color = 'green'
    else:
        print(f'On head 2, epoch {epoch}...')
        losses, accuracies, val_accuracies_1, val_accuracies_2 = train_100labels(model, device, loader_head2, optimizer, "head2", val_loader1 = loader_head1_test, val_loader2 = loader_head2_test)
        head = 2
        color = 'orange'

    all_losses.extend(losses)
    all_accuracies.extend(accuracies)
    all_accuracies_val1.extend(val_accuracies_1)
    all_accuracies_val2.extend(val_accuracies_2)
    batch_heads.extend([head] * len(losses))
    colors.extend([color] * len(losses))
    if epoch % 30 == 0:
      torch.save({
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
      }, drive_results_path + f'/{experiment_name}_checkpoint{epoch}.pth')

  data = {
      'all_losses': all_losses,
      'all_accuracies': all_accuracies,
      'all_accuracies_val1': all_accuracies_val1,
      'all_accuracies_val2': all_accuracies_val2,
      'batch_heads': batch_heads,
      'colors': colors
  }

  # Define the file path
  file_path = drive_results_path + f'/{experiment_name}.json'

  # Write the dictionary to a file as JSON
  with open(file_path, 'w') as file:
      json.dump(data, file)

# Repetitions with Resnet18

## 1) CIFAR5 + CIFAR5 (paper)  

In [ ]:
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare datasets
train_dataset = datasets.CIFAR10('./CIFAR10', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
                                ]))

# Prepare the CIFAR10 test dataset
test_dataset = datasets.CIFAR10('./CIFAR10', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
                                ]))

# Separate datasets for each head
indices_head1 = [i for i, (_, target) in enumerate(train_dataset) if target < 5]
indices_head2 = [i for i, (_, target) in enumerate(train_dataset) if target >= 5]

indices_head1_test = [i for i, (_, target) in enumerate(test_dataset) if target < 5]
indices_head2_test = [i for i, (_, target) in enumerate(test_dataset) if target >= 5]


# The first dataset includes the classes for the first head, and the second dataset for the second head
dataset_head1 = Subset(train_dataset, indices_head1)
dataset_head2 = Subset(train_dataset, indices_head2)

dataset_head1_test = Subset(test_dataset, indices_head1_test)
dataset_head2_test = Subset(test_dataset, indices_head2_test)

bs = 512
loader_head1 = DataLoader(dataset_head1, batch_size=bs, shuffle=True)
loader_head2 = DataLoader(dataset_head2, batch_size=bs, shuffle=True)

loader_head1_test = DataLoader(dataset_head1_test, batch_size=bs, shuffle=True)
loader_head2_test = DataLoader(dataset_head2_test, batch_size=bs, shuffle=True)

100%|██████████| 170498071/170498071 [00:13<00:00, 13084343.08it/s]


Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


In [ ]:
# Define how many epochs to run for each head before switching
run_lengths = [3,6,15]
total_epochs = 90*2  # Total number of epochs to train

for run_length in run_lengths:
  print(f'***Running for run_length={run_length}***')
  all_losses = []
  all_accuracies = []
  all_accuracies_val1 = [] # Validation output for head 1
  all_accuracies_val2 = [] # Validation output for head 2
  batch_heads = []
  colors = []
  model = ResNet18TwoHeads(num_classes1=5, num_classes2=5).to(device)
  #optimizer = Adam(model.parameters())
  optimizer_name = 'SGD'
  optimizer = SGD(model.parameters(),lr=0.02, momentum=0.0)
  experiment_name = f'cifar5_cifar5/rl{run_length}_bs{bs}_epochs{total_epochs//2}_{optimizer_name}_lr{optimizer.param_groups[0]["lr"]}_momentum{optimizer.param_groups[0]["momentum"]}'
  if run_length == 0: # Reserve 0 for iid setting
    training_loop_iid(model, optimizer, run_length, total_epochs//2, experiment_name)
  else:
    training_loop(model, optimizer, run_length, total_epochs, experiment_name)


***Running for run_length=3***
On head 1, epoch 1...


Training Progress: 100%|██████████| 49/49 [00:15<00:00,  3.07it/s]


On head 1, epoch 2...


Training Progress: 100%|██████████| 49/49 [00:16<00:00,  3.04it/s]


On head 1, epoch 3...


...

Training Progress: 100%|██████████| 49/49 [00:16<00:00,  2.98it/s]


On head 2, epoch 179...


Training Progress: 100%|██████████| 49/49 [00:16<00:00,  3.01it/s]


On head 2, epoch 180...


Training Progress: 100%|██████████| 49/49 [00:16<00:00,  3.06it/s]


## 2) STL5 + STL5 (alternative)


In [ ]:
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare the STL10 dataset
train_dataset = datasets.STL10('./STL10', split='train', download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  # Normalize using mean and std of STL10 dataset
                                  transforms.Normalize(mean=[0.43, 0.42, 0.39], std=[0.27, 0.26, 0.27])
                              ]))
test_dataset = datasets.STL10('./STL10', split='test', download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  # Normalize using mean and std of STL10 dataset
                                  transforms.Normalize(mean=[0.43, 0.42, 0.39], std=[0.27, 0.26, 0.27])
                              ]))


# Separate datasets for each head
indices_head1 = [i for i, (_, target) in enumerate(train_dataset) if target < 5]
indices_head2 = [i for i, (_, target) in enumerate(train_dataset) if target >= 5]

indices_head1_test = [i for i, (_, target) in enumerate(test_dataset) if target < 5]
indices_head2_test = [i for i, (_, target) in enumerate(test_dataset) if target >= 5]


# The first dataset includes the classes for the first head, and the second dataset for the second head
dataset_head1 = Subset(train_dataset, indices_head1)
dataset_head2 = Subset(train_dataset, indices_head2)

dataset_head1_test = Subset(test_dataset, indices_head1_test)
dataset_head2_test = Subset(test_dataset, indices_head2_test)

bs = 64
loader_head1 = DataLoader(dataset_head1, batch_size=bs, shuffle=True)
loader_head2 = DataLoader(dataset_head2, batch_size=bs, shuffle=True)

val_bs = 256
loader_head1_test = DataLoader(dataset_head1_test, batch_size=val_bs, shuffle=True)
loader_head2_test = DataLoader(dataset_head2_test, batch_size=val_bs, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define how many epochs to run for each head before switching
run_lengths = [3,6,15]
total_epochs = 90*2  # Total number of epochs to train

for run_length in run_lengths:
  print(f'***Running for run_length={run_length}***')
  all_losses = []
  all_accuracies = []
  all_accuracies_val1 = [] # Validation output for head 1
  all_accuracies_val2 = [] # Validation output for head 2
  batch_heads = []
  colors = []
  model = ResNet18TwoHeads(num_classes1=5, num_classes2=5).to(device)
  #optimizer = Adam(model.parameters())
  optimizer_name = 'SGD'
  optimizer = SGD(model.parameters(),lr=0.02, momentum=0.0)
  experiment_name = f'stl5_stl5/rl{run_length}_bs{bs}_epochs{total_epochs//2}_{optimizer_name}_lr{optimizer.param_groups[0]["lr"]}_momentum{optimizer.param_groups[0]["momentum"]}'
  if run_length == 0: # Reserve 0 for iid setting
    training_loop_iid(model, optimizer, run_length, total_epochs//2, experiment_name)
  else:
    training_loop(model, optimizer, run_length, total_epochs, experiment_name)

***Running for run_length=3***
On head 1, epoch 1...


Training Progress: 100%|██████████| 40/40 [00:13<00:00,  2.92it/s]


On head 1, epoch 2...


Training Progress:  68%|██████▊   | 27/40 [00:10<00:02,  5.33it/s]

##3) MNIST5 + FashionMNIST5

In [17]:
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare datasets
train_dataset1 = datasets.MNIST('./MNIST', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))

# Prepare the FashionMNIST dataset with normalization
train_dataset2 = datasets.FashionMNIST('./FashionMNIST', train=True, download=True,
                                     transform=transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.2860], std=[0.3530])
                                     ]))

# Prepare datasets
test_dataset1 = datasets.MNIST('./MNIST', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))

# Prepare the FashionMNIST dataset with normalization
test_dataset2 = datasets.FashionMNIST('./FashionMNIST', train=False, download=True,
                                     transform=transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.2860], std=[0.3530])
                                     ]))

# Separate datasets for each head
indices_head1 = [i for i, (_, target) in enumerate(train_dataset1) if target < 5]
indices_head2 = [i for i, (_, target) in enumerate(train_dataset2) if target >= 5]

indices_head1_test = [i for i, (_, target) in enumerate(test_dataset1) if target < 5]
indices_head2_test = [i for i, (_, target) in enumerate(test_dataset2) if target >= 5]


# The first dataset includes the classes for the first head, and the second dataset for the second head
dataset_head1 = Subset(train_dataset1, indices_head1)
dataset_head2 = Subset(train_dataset2, indices_head2)

dataset_head1_test = Subset(test_dataset1, indices_head1_test)
dataset_head2_test = Subset(test_dataset2, indices_head2_test)

bs = 512
loader_head1 = DataLoader(dataset_head1, batch_size=bs, shuffle=True)
loader_head2 = DataLoader(dataset_head2, batch_size=bs, shuffle=True)

val_bs = 512
loader_head1_test = DataLoader(dataset_head1_test, batch_size=val_bs, shuffle=True)
loader_head2_test = DataLoader(dataset_head2_test, batch_size=val_bs, shuffle=True)

In [18]:
# Define how many epochs to run for each head before switching
run_lengths = [3,6,15]
total_epochs = 90*2  # Total number of epochs to train

for run_length in run_lengths:
  print(f'***Running for run_length={run_length}***')
  all_losses = []
  all_accuracies = []
  all_accuracies_val1 = [] # Validation output for head 1
  all_accuracies_val2 = [] # Validation output for head 2
  batch_heads = []
  colors = []
  model = ResNet18TwoHeads_MNIST().to(device)
  #optimizer = Adam(model.parameters())
  optimizer_name = 'SGD'
  optimizer = SGD(model.parameters(),lr=0.02, momentum=0.0)
  experiment_name = f'mnist5_fashionmnist5/rl{run_length}_bs{bs}_epochs{total_epochs//2}_{optimizer_name}_lr{optimizer.param_groups[0]["lr"]}_momentum{optimizer.param_groups[0]["momentum"]}'
  if run_length == 0: # Reserve 0 for iid setting
    training_loop_iid(model, optimizer, run_length, total_epochs//2, experiment_name)
  else:
    training_loop(model, optimizer, run_length, total_epochs, experiment_name)

***Running for run_length=3***
On head 1, epoch 1...


Training Progress: 100%|██████████| 60/60 [00:17<00:00,  3.46it/s]


On head 1, epoch 2...


Training Progress: 100%|██████████| 60/60 [00:16<00:00,  3.65it/s]


On head 1, epoch 3...


...

Training Progress: 100%|██████████| 59/59 [00:16<00:00,  3.53it/s]


On head 2, epoch 179...


Training Progress: 100%|██████████| 59/59 [00:16<00:00,  3.56it/s]


On head 2, epoch 180...


Training Progress: 100%|██████████| 59/59 [00:17<00:00,  3.44it/s]


## 4) CIFAR50 + CIFAR50

In [ ]:
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare CIFAR100 dataset
train_dataset = datasets.CIFAR100('./CIFAR100', train=True, download=True,
                                  transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5071, 0.4867, 0.4408],
                                                           std=[0.2675, 0.2565, 0.2761])
                                  ]))

test_dataset = datasets.CIFAR100('./CIFAR100', train=False, download=True,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5071, 0.4867, 0.4408],
                                                          std=[0.2675, 0.2565, 0.2761])
                                 ]))

# Separate datasets for each head based on class indices
indices_head1 = [i for i, (_, target) in enumerate(train_dataset) if target < 50]
indices_head2 = [i for i, (_, target) in enumerate(train_dataset) if target >= 50]

indices_head1_test = [i for i, (_, target) in enumerate(test_dataset) if target < 50]
indices_head2_test = [i for i, (_, target) in enumerate(test_dataset) if target >= 50]

# Creating subsets for each head for both training and testing
dataset_head1 = Subset(train_dataset, indices_head1)
dataset_head2 = Subset(train_dataset, indices_head2)

dataset_head1_test = Subset(test_dataset, indices_head1_test)
dataset_head2_test = Subset(test_dataset, indices_head2_test)

# Set the batch size
bs = 512

# Creating dataloaders for both training and testing
loader_head1 = DataLoader(dataset_head1, batch_size=bs, shuffle=True)
loader_head2 = DataLoader(dataset_head2, batch_size=bs, shuffle=True)

val_bs = 1024
loader_head1_test = DataLoader(dataset_head1_test, batch_size=val_bs, shuffle=True)
loader_head2_test = DataLoader(dataset_head2_test, batch_size=val_bs, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define how many epochs to run for each head before switching
run_lengths = [3,6,10,15]
total_epochs = 120*2  # Total number of epochs to train

for run_length in run_lengths:
  print(f'***Running for run_length={run_length}***')
  all_losses = []
  all_accuracies = []
  all_accuracies_val1 = [] # Validation output for head 1
  all_accuracies_val2 = [] # Validation output for head 2
  batch_heads = []
  colors = []
  model = ResNet18TwoHeads().to(device)
  #optimizer = Adam(model.parameters())
  optimizer_name = 'SGD'
  optimizer = SGD(model.parameters(),lr=0.02, momentum=0.0)
  experiment_name = f'cifar50_cifar50/rl{run_length}_bs{bs}_epochs{total_epochs//2}_{optimizer_name}_lr{optimizer.param_groups[0]["lr"]}_momentum{optimizer.param_groups[0]["momentum"]}'
  if run_length == 0: # Reserve 0 for iid setting
    #training_loop_iid(model, optimizer, run_length, total_epochs//2, experiment_name)
    pass
  else:
    training_loop_100labels(model, optimizer, run_length, total_epochs, experiment_name)

***Running for run_length=3***
On head 1, epoch 1...


Training Progress: 100%|██████████| 49/49 [00:17<00:00,  2.76it/s]


On head 1, epoch 2...


Training Progress: 100%|██████████| 49/49 [00:18<00:00,  2.71it/s]


On head 1, epoch 3...


...

Training Progress: 100%|██████████| 49/49 [00:18<00:00,  2.62it/s]


On head 2, epoch 239...


Training Progress: 100%|██████████| 49/49 [00:17<00:00,  2.76it/s]


On head 2, epoch 240...


Training Progress: 100%|██████████| 49/49 [00:17<00:00,  2.74it/s]
